# Learning pendulum inverse dynamics

In [1]:
import os
import pandas as pd
import numpy as np
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import sys

sys.path.append("../..")

from src.physics_dataset import PhysicsStatesDataset
from src.ai_models import FullyConnectedCtrl, CasadiModel

In [2]:
dataset = PhysicsStatesDataset("../../data/interim/pendulum/mixed1.csv")

In [3]:
dataset.dataframe.head()

,id,time,qpos,qvel,qacc,ctrl
0,0,0.001,[-5.27405987e-05],[-0.0527406],[-52.74059872],[-57.4872526]
1,1,0.002,[-0.00013264],[-0.07989561],[-27.15500922],[-29.59911526]
2,2,0.003,[-0.00021852],[-0.08588338],[-5.9877766],[-6.52706684]
3,3,0.004,[-0.00031071],[-0.09218989],[-6.30650874],[-6.87473762]
4,4,0.005,[-0.00044756],[-0.13684961],[-44.65972047],[-48.68000974]


In [4]:
from sklearn.model_selection import train_test_split

proportion = 0.2
x_train, x_test, y_train, y_test = train_test_split(
    dataset.states, dataset.ctrls, test_size=proportion, random_state=42
)

In [5]:
train_loader = DataLoader(
    TensorDataset(torch.tensor(x_train), torch.tensor(y_train)),
    batch_size=20,
    shuffle=True,
)
val_loader = DataLoader(
    TensorDataset(torch.tensor(x_test), torch.tensor(y_test)),
    batch_size=20,
    shuffle=True,
)

/tmp/ipykernel_221631/3394741560.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  TensorDataset(torch.tensor(x_train), torch.tensor(y_train)),
/tmp/ipykernel_221631/3394741560.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  TensorDataset(torch.tensor(x_test), torch.tensor(y_test)),


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = FullyConnectedCtrl(
    nv=1,
    ctrl=1,
    linear_layers=3,
    linear_hidden_size=100,
).to(device)

In [7]:
model.load_state_dict(torch.load("../../data/models/pendulum_fc.pt"))

<All keys matched successfully>

In [8]:
# _, val_losses = model.train_model(
#     train_loader,
#     val_loader,
#     epochs=30,
#     ckpt_path="../../data/models/pendulum_fc.pt",
# )


In [9]:
# plt.title("Validation losses")
# plt.plot(val_losses)
# plt.grid()
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.show()

### Model is trained, now we go to inferecing and copying its weights

In [13]:
model(torch.tensor([0, 0, 0], dtype=torch.float))

tensor([-0.3516], grad_fn=<ViewBackward0>)

In [10]:
casadi_model = CasadiModel(model)

In [11]:
casadi_model(np.array([0, 0, 0]))

DM(-0.351554)

As we see, we have transfered all weights correctly